In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

import statsmodels as sms
import statsmodels.formula.api as smf

import seaborn as sns # for data visualization
sns.set_style("whitegrid")

from dateutil.relativedelta import *
from pandas.tseries.offsets import *

pd.set_option('display.max_columns', None)

DATA_FOLDER = '/Users/mmw/data/misp_data'

In [3]:
ret_decomp_dcf5 = pd.read_csv(f'{DATA_FOLDER}/decomp_dcf5y_RF_1970_2019.csv', index_col=0).replace([np.inf, -np.inf], np.nan).dropna()

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
ret_decomp_dcf5.head()

,vt,permno,ticker,me,year,jdate_crsp,logret_3,logret_6,logret_24,cumret_3,cumret_6,cumret_24,logret_12,logret_36,logret_60,cumret_12,cumret_36,cumret_60,termlogret,termcumret,vt+3,me+3,vt+6,me+6,vt+12,me+12,vt+24,me+24,vt+36,me+36,vt+60,me+60,r_t+3,r*_t+3,r_unchg+3,r_chg+3,r_t+6,r*_t+6,r_unchg+6,r_chg+6,r_t+12,r*_t+12,r_unchg+12,r_chg+12,r_t+24,r*_t+24,r_unchg+24,r_chg+24,r_t+36,r*_t+36,r_unchg+36,r_chg+36,r_t+60,r*_t+60,r_unchg+60,r_chg+60,misp
61,10.127232,10001,EGAS,11.323983,2012,2012-07-31,-0.002529,-0.051853,0.151883,-0.002526,-0.050532,0.164024,0.073598,0.165520,0.483474,0.076374,0.180006,0.621699,3.332619,27.011595,10.127232,11.311315,10.120592,11.296132,10.120592,11.568660,10.123883,11.814439,10.121637,11.564044,10.109022,11.822129,-0.002529,0.0,-0.029919,-0.012351,-0.051853,-0.00664,-0.029919,-0.020681,0.073598,-0.00664,-0.029919,0.245034,0.151883,-0.003349,-0.029919,0.481460,0.165520,-0.005595,-0.029919,0.239515,0.483474,-0.018209,-0.029919,0.503447,1.196751
60,10.127232,10001,EGAS,11.319045,2012,2012-06-30,-0.030735,-0.064210,0.115162,-0.030268,-0.062192,0.122056,0.039589,0.108256,0.432422,0.040384,0.114333,0.540985,3.332619,27.011595,10.127232,11.303199,10.127232,11.239867,10.120592,11.397112,10.123883,11.613165,10.121637,11.590608,10.109022,11.820197,-0.030735,0.0,-0.029795,-0.015450,-0.064210,0.00000,-0.029795,-0.077198,0.039589,-0.00664,-0.029795,0.082589,0.115162,-0.003349,-0.029795,0.290032,0.108256,-0.005595,-0.029795,0.270229,0.432422,-0.018209,-0.029795,0.506378,1.191813
59,10.127232,10001,EGAS,11.351195,2012,2012-05-31,-0.106477,-0.105771,0.061089,-0.101004,-0.100369,0.062993,-0.048060,0.027098,0.328415,-0.046923,0.027469,0.388765,3.332619,27.011595,10.127232,11.307215,10.127232,11.260027,10.120592,11.364996,10.123883,11.608409,10.121637,11.560572,10.109022,11.802635,-0.106477,0.0,-0.030599,-0.042881,-0.105771,0.00000,-0.030599,-0.088890,-0.048060,-0.00664,-0.030599,0.019929,0.061089,-0.003349,-0.030599,0.254049,0.027098,-0.005595,-0.030599,0.209598,0.328415,-0.018209,-0.030599,0.457908,1.223964
58,10.127232,10001,EGAS,11.443511,2012,2012-04-30,-0.130952,-0.133481,-0.038340,-0.122740,-0.124956,-0.037614,-0.080185,0.001001,0.329175,-0.077054,0.001001,0.389821,3.332619,27.011595,10.127232,11.323983,10.127232,11.311315,10.120592,11.369848,10.123883,11.646750,10.121637,11.564109,10.109022,11.786762,-0.130952,0.0,-0.032907,-0.116540,-0.133481,0.00000,-0.032907,-0.128892,-0.080185,-0.00664,-0.032907,-0.065348,-0.038340,-0.003349,-0.032907,0.201423,0.001001,-0.005595,-0.032907,0.123039,0.329175,-0.018209,-0.032907,0.352424,1.316279
57,10.127232,10001,EGAS,11.462428,2012,2012-03-31,-0.055638,-0.086374,-0.067317,-0.054119,-0.082748,-0.065101,-0.062799,0.049104,0.366950,-0.060868,0.050330,0.443326,3.332619,27.011595,10.127232,11.319045,10.127232,11.303199,10.120592,11.329019,10.123883,11.563116,10.121637,11.553126,10.109022,11.802635,-0.055638,0.0,-0.033380,-0.139799,-0.086374,0.00000,-0.033380,-0.155249,-0.062799,-0.00664,-0.033380,-0.123600,-0.067317,-0.003349,-0.033380,0.101436,0.049104,-0.005595,-0.033380,0.093886,0.366950,-0.018209,-0.033380,0.349456,1.335196


In [5]:
ret_decomp_dcf5.groupby(pd.qcut(ret_decomp_dcf5.me, 10)).agg('mean')

,vt,permno,me,year,logret_3,logret_6,logret_24,cumret_3,cumret_6,cumret_24,logret_12,logret_36,logret_60,cumret_12,cumret_36,cumret_60,termlogret,termcumret,vt+3,me+3,vt+6,me+6,vt+12,me+12,vt+24,me+24,vt+36,me+36,vt+60,me+60,r_t+3,r*_t+3,r_unchg+3,r_chg+3,r_t+6,r*_t+6,r_unchg+6,r_chg+6,r_t+12,r*_t+12,r_unchg+12,r_chg+12,r_t+24,r*_t+24,r_unchg+24,r_chg+24,r_t+36,r*_t+36,r_unchg+36,r_chg+36,r_t+60,r*_t+60,r_unchg+60,r_chg+60,misp
me,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"(2.888, 9.36]",9.997823,47636.454148,8.590686,1986.396184,0.019898,0.059123,0.261557,0.069062,0.162548,0.715649,0.136664,0.361554,0.497313,0.358865,1.075781,1.899766,-0.124127,19.512957,9.998713,8.643175,9.999784,8.692338,10.001549,8.792664,10.007758,8.962077,10.012678,9.108568,10.025773,9.330367,0.019898,0.000890,0.035178,0.050308,0.059123,0.001961,0.035178,0.097198,0.136664,0.003726,0.035178,0.193296,0.261557,0.009936,0.035178,0.352420,0.361554,0.014855,0.035178,0.490451,0.497313,0.027951,0.035178,0.693937,-1.407137
"(9.36, 10.176]",10.015193,49075.746484,9.788433,1989.503425,0.015298,0.038212,0.157817,0.051555,0.112926,0.500884,0.082830,0.222460,0.332460,0.247165,0.756472,1.303057,0.676413,47.267106,10.016100,9.814601,10.017075,9.842022,10.019316,9.897949,10.027691,9.997866,10.038165,10.088776,10.067909,10.248591,0.015298,0.000907,0.005669,0.024630,0.038212,0.001882,0.005669,0.050414,0.082830,0.004123,0.005669,0.102757,0.157817,0.012499,0.005669,0.192011,0.222460,0.022973,0.005669,0.270435,0.332460,0.052717,0.005669,0.397255,-0.226759
"(10.176, 10.788]",10.055098,51783.174649,10.490330,1991.412167,0.015779,0.034134,0.143055,0.046928,0.098062,0.411202,0.074636,0.194249,0.290242,0.209663,0.611415,1.094365,1.081539,61.150704,10.056142,10.510588,10.057488,10.532880,10.062166,10.580206,10.080955,10.663203,10.102518,10.728155,10.157509,10.855651,0.015779,0.001044,-0.010881,0.018733,0.034134,0.002389,-0.010881,0.039156,0.074636,0.007067,-0.010881,0.080739,0.143055,0.025857,-0.010881,0.143342,0.194249,0.047419,-0.010881,0.185646,0.290242,0.102411,-0.010881,0.256338,0.435231
"(10.788, 11.321]",10.107111,53945.005401,11.058440,1993.122049,0.017444,0.034793,0.138771,0.045013,0.092042,0.381268,0.071629,0.195562,0.288918,0.190666,0.573156,1.050751,1.386907,78.387777,10.108993,11.077425,10.111916,11.097615,10.121656,11.140540,10.155804,11.217917,10.198151,11.286614,10.283998,11.406611,0.017444,0.001882,-0.023783,0.016676,0.034793,0.004805,-0.023783,0.033511,0.071629,0.014545,-0.023783,0.065867,0.138771,0.048693,-0.023783,0.108014,0.195562,0.091040,-0.023783,0.133705,0.288918,0.176886,-0.023783,0.167003,0.951328
"(11.321, 11.823]",10.227062,55178.125445,11.571853,1994.108570,0.016704,0.031469,0.115665,0.042019,0.083190,0.329520,0.061605,0.165354,0.260892,0.172416,0.495134,0.912090,1.793182,99.279320,10.232426,11.587748,10.238983,11.603911,10.256450,11.633913,10.310597,11.690087,10.370757,11.740986,10.482090,11.849799,0.016704,0.005365,-0.033620,0.010267,0.031469,0.011922,-0.033620,0.019632,0.061605,0.029388,-0.033620,0.031855,0.115665,0.083535,-0.033620,0.033831,0.165354,0.143696,-0.033620,0.024801,0.260892,0.255029,-0.033620,0.022344,1.344792
"(11.823, 12.316]",10.417331,56281.638445,12.066205,1995.205572,0.017196,0.029938,0.104675,0.039790,0.074896,0.288828,0.054521,0.154565,0.263977,0.145947,0.433979,0.828734,2.076880,107.074394,10.426539,12.079211,10.436311,12.091303,10.461305,12.115012,10.528685,12.164045,10.602390,12.210517,10.726982,12.318187,0.017196,0.009208,-0.041222,0.003704,0.029938,0.018980,-0.041222,0.005966,0.054521,0.043974,-0.041222,0.004713,0.104675,0.111355,-0.041222,-0.013176,0.154565,0.185059,-0.041222,-0.039728,0.263977,0.309651,-0.041222,-0.056227,1.648874
"(12.316, 12.86]",10.734657,56795.071692,12.581929,1996.500801,0.017071,0.027442,0.086436,0.037811,0.067926,0.252029,0.047630,0.139299,0.245439,0.130103,0.393868,0.752263,2.328707,139.853543,10.748664,12.591402,10.764283,12.600777,10.795951,12.616150,10.866615,12.646898,10.926337,12

In [6]:
ret_decomp_dcf5.groupby(pd.qcut(ret_decomp_dcf5.me, 10)).agg('std')

,vt,permno,me,year,logret_3,logret_6,logret_24,cumret_3,cumret_6,cumret_24,logret_12,logret_36,logret_60,cumret_12,cumret_36,cumret_60,termlogret,termcumret,vt+3,me+3,vt+6,me+6,vt+12,me+12,vt+24,me+24,vt+36,me+36,vt+60,me+60,r_t+3,r*_t+3,r_unchg+3,r_chg+3,r_t+6,r*_t+6,r_unchg+6,r_chg+6,r_t+12,r*_t+12,r_unchg+12,r_chg+12,r_t+24,r*_t+24,r_unchg+24,r_chg+24,r_t+36,r*_t+36,r_unchg+36,r_chg+36,r_t+60,r*_t+60,r_unchg+60,r_chg+60,misp
me,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"(2.888, 9.36]",0.317774,21728.971268,0.636573,9.932029,0.293531,0.403924,0.715944,0.405245,0.678857,2.017021,0.547496,0.840032,1.087829,1.151989,2.825640,9.717378,2.776758,163.030280,0.319244,0.688412,0.320942,0.736353,0.325579,0.819156,0.332787,0.932670,0.342156,1.023497,0.371448,1.193994,0.293531,0.043084,0.017838,0.320250,0.403924,0.063767,0.017838,0.439118,0.547496,0.100079,0.017838,0.598369,0.715944,0.113133,0.017838,0.769917,0.840032,0.134844,0.017838,0.890137,1.087829,0.157441,0.017838,1.089157,0.713517
"(9.36, 10.176]",0.357174,23451.997553,0.234192,11.213252,0.258846,0.361221,0.680720,0.317653,0.494778,3.157468,0.500969,0.820366,1.079968,1.925123,4.120162,3.228005,2.828595,418.392237,0.358919,0.358545,0.360939,0.446307,0.364328,0.578573,0.377353,0.752148,0.395010,0.875660,0.437612,1.069775,0.258846,0.048109,0.010560,0.269129,0.361221,0.073176,0.010560,0.376749,0.500969,0.102719,0.010560,0.523627,0.680720,0.126183,0.010560,0.698239,0.820366,0.153375,0.010560,0.817761,1.079968,0.207668,0.010560,0.987866,0.422401
"(10.176, 10.788]",0.470707,24572.906034,0.175830,11.810470,0.241670,0.339357,0.648006,0.299742,0.447057,1.109699,0.474445,0.787382,1.047993,0.731680,1.525435,2.488894,2.871121,441.991230,0.472259,0.307447,0.474488,0.395410,0.480358,0.528065,0.500918,0.698647,0.516787,0.821308,0.567884,1.018352,0.241670,0.065737,0.012443,0.253689,0.339357,0.087515,0.012443,0.354159,0.474445,0.122624,0.012443,0.495947,0.648006,0.164170,0.012443,0.652707,0.787382,0.209237,0.012443,0.757949,1.047993,0.286494,0.012443,0.910220,0.497731
"(10.788, 11.321]",0.503577,24992.578565,0.153585,11.845017,0.228929,0.323951,0.629855,0.263386,0.411932,0.980708,0.454155,0.764442,1.033591,0.646748,1.359187,2.297878,2.820171,617.814104,0.505877,0.283018,0.509373,0.371454,0.520207,0.498774,0.552044,0.671578,0.590775,0.788336,0.669158,1.004457,0.228929,0.082563,0.012978,0.244452,0.323951,0.112442,0.012978,0.344773,0.454155,0.164966,0.012978,0.477420,0.629855,0.234816,0.012978,0.629592,0.764442,0.297265,0.012978,0.713967,1.033591,0.412045,0.012978,0.857225,0.519106
"(11.321, 11.823]",0.626945,24991.806772,0.146069,11.876375,0.219691,0.310775,0.607277,0.260555,0.396274,0.893558,0.440720,0.734555,0.986175,1.403867,1.163214,2.072343,2.784309,516.609912,0.630435,0.273362,0.635758,0.357464,0.649563,0.484459,0.694929,0.646623,0.739727,0.761211,0.825862,0.967308,0.219691,0.113364,0.015853,0.247374,0.310775,0.162331,0.015853,0.346470,0.440720,0.228736,0.015853,0.478245,0.607277,0.318271,0.015853,0.606763,0.734555,0.392049,0.015853,0.681539,0.986175,0.512967,0.015853,0.809513,0.634135
"(11.823, 12.316]",0.744725,25223.826313,0.142438,12.042696,0.209876,0.294874,0.576546,0.229989,0.340703,0.777025,0.415001,0.693028,0.927843,0.520195,0.997751,1.738930,2.653598,528.174798,0.750168,0.261517,0.756078,0.338507,0.773025,0.454722,0.817817,0.609982,0.869779,0.720040,0.958662,0.918860,0.209876,0.146951,0.018609,0.253275,0.294874,0.206620,0.018609,0.348995,0.415001,0.294862,0.018609,0.475536,0.576546,0.402539,0.018609,0.595125,0.693028,0.484044,0.018609,0.655563,0.927843,0.607567,0.018609,0.767920,0.744373
"(12.316, 12.86]",0.923950,25851.252458,0.156579,11.990711,0.201534,0.282469,0.563173,0.221278,0.308298,0.684201,0.401043,0.673916,0.894956,0.463861,0.915805,1.571889,2.623345,668.532524,0.927783,0.262156,0.933669,0.333929,0.946976,0.446691,0.991255,0.594801,1.035276,0.695246,1.112928,0.884002,0.201534,0.188772,0.022868,0.272545,0.282469,0.268345,0.022868,0.377681,0.401043,0.379418,0.022

In [8]:
ret_decomp_dcf5_hist = pd.read_csv(f'{DATA_FOLDER}/decomp_dcf5y_hist_1960_2014.csv', index_col=0).replace([np.inf, -np.inf], np.nan).dropna()

/Users/mmw/Installations/venvs/mlval/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [11]:
ret_decomp_dcf5 = ret_decomp_dcf5_hist[['be', 'jdate_crsp', 'permno']].merge(ret_decomp_dcf5, on=[ 'jdate_crsp', 'permno'], how='inner')

In [12]:
ret_decomp_dcf5['bm'] = np.exp(np.log(ret_decomp_dcf5['be']) - ret_decomp_dcf5['me'])

In [13]:
ret_decomp_dcf5.groupby(pd.qcut(ret_decomp_dcf5.bm, 10)).agg('mean')

,be,permno,vt,me,year,logret_3,logret_6,logret_24,cumret_3,cumret_6,cumret_24,logret_12,logret_36,logret_60,cumret_12,cumret_36,cumret_60,termlogret,termcumret,vt+3,me+3,vt+6,me+6,vt+12,me+12,vt+24,me+24,vt+36,me+36,vt+60,me+60,r_t+3,r*_t+3,r_unchg+3,r_chg+3,r_t+6,r*_t+6,r_unchg+6,r_chg+6,r_t+12,r*_t+12,r_unchg+12,r_chg+12,r_t+24,r*_t+24,r_unchg+24,r_chg+24,r_t+36,r*_t+36,r_unchg+36,r_chg+36,r_t+60,r*_t+60,r_unchg+60,r_chg+60,misp,bm
bm,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"(-0.000923, 0.223]",1.726716e+06,49368.158814,12.056828,14.004390,1995.568935,0.035514,0.052401,0.145618,0.057503,0.096531,0.333430,0.082053,0.225957,0.406680,0.174396,0.522021,0.988614,4.363167,686.468859,12.109668,14.017120,12.154380,14.032861,12.231927,14.055273,12.350196,14.106313,12.430627,14.168841,12.611072,14.309085,0.035514,0.052841,-0.048689,-0.039108,0.052401,0.097552,-0.048689,-0.067354,0.082053,0.175100,-0.048689,-0.121111,0.145618,0.293368,-0.048689,-0.186659,0.225957,0.373799,-0.048689,-0.204115,0.406680,0.554245,-0.048689,-0.243311,1.947562,0.143483
"(0.223, 0.335]",1.953342e+06,48693.224100,11.890545,13.615805,1994.076673,0.030555,0.049506,0.170380,0.049203,0.086743,0.338995,0.089739,0.260235,0.468239,0.167139,0.528191,1.017409,4.291910,593.203540,11.927394,13.633143,11.963512,13.652029,12.029826,13.689715,12.122608,13.760566,12.197156,13.838255,12.354108,14.014015,0.030555,0.036850,-0.043132,-0.019024,0.049506,0.072968,-0.043132,-0.035825,0.089739,0.139281,-0.043132,-0.063737,0.170380,0.232064,-0.043132,-0.085121,0.260235,0.306611,-0.043132,-0.082057,0.468239,0.463564,-0.043132,-0.063720,1.725260,0.280541
"(0.335, 0.44]",1.841410e+06,48922.455678,11.656267,13.215796,1993.132125,0.031600,0.055380,0.200673,0.048977,0.091002,0.369237,0.103850,0.302938,0.525011,0.177674,0.579252,1.100312,4.038054,468.211575,11.684883,13.237074,11.714345,13.260320,11.767983,13.303420,11.858171,13.390877,11.943242,13.482162,12.093250,13.676168,0.031600,0.028616,-0.038988,-0.007154,0.055380,0.058078,-0.038988,-0.013215,0.103850,0.111716,-0.038988,-0.023489,0.200673,0.201904,-0.038988,-0.026152,0.302938,0.286975,-0.038988,-0.020094,0.525011,0.436983,-0.038988,0.022804,1.559529,0.387055
"(0.44, 0.554]",1.335718e+06,48579.590482,11.391798,12.778404,1992.272381,0.030192,0.055150,0.210395,0.047952,0.091172,0.385911,0.108585,0.318424,0.552514,0.185621,0.616391,1.179214,3.759925,361.535219,11.415445,12.800539,11.437619,12.824637,11.481482,12.872140,11.562956,12.963586,11.639121,13.059922,11.796824,13.271808,0.030192,0.023647,-0.034665,-0.001474,0.055150,0.045821,-0.034665,0.000402,0.108585,0.089683,-0.034665,0.003951,0.210395,0.171158,-0.034665,0.013674,0.318424,0.247323,-0.034665,0.033340,0.552514,0.405026,-0.034665,0.086168,1.386606,0.496308
"(0.554, 0.677]",1.430938e+06,46940.196035,11.313212,12.527790,1991.431720,0.028715,0.057665,0.225886,0.045349,0.092427,0.394147,0.113382,0.344741,0.597732,0.188089,0.638864,1.266068,3.729903,295.935513,11.331635,12.551703,11.350127,12.577426,11.388471,12.627928,11.455875,12.729298,11.532266,12.833777,11.685898,13.058883,0.028715,0.018423,-0.030364,0.005353,0.057665,0.036915,-0.030364,0.012404,0.113382,0.075260,-0.030364,0.024257,0.225886,0.142663,-0.030364,0.057373,0.344741,0.219054,-0.030364,0.084760,0.597732,0.372686,-0.030364,0.154447,1.214578,0.614343
"(0.677, 0.819]",1.327938e+06,44899.800159,11.202109,12.258569,1990.147168,0.028742,0.058436,0.242844,0.044797,0.092144,0.414670,0.119738,0.367158,0.628497,0.193723,0.673759,1.326414,3.659536,249.502033,11.215337,12.283237,11.228055,12.309805,11.257642,12.363734,11.322971,12.471207,11.396550,12.576658,11.550921,12.805724,0.028742,0.013227,-0.026411,0.011155,0.058436,0.025946,-0.026411,0.024658,0.119738,0.055532,-0.026411,0.048392,0.242844,0.120861,-0.026411,0.089482,0.367158,0.194441,-0.026411,0.120558,0.628497,0.348812,-0.026411,0.193385,1.056459,0.746406
"(0.819, 0.994]",1.195772e+06,42323.421216,11.149565,12.043580,1988.361288,0.030036,0.064987,0.278731,0.

In [14]:
ret_decomp_dcf5.groupby(pd.qcut(ret_decomp_dcf5.bm, 10)).agg('std')

,be,permno,vt,me,year,logret_3,logret_6,logret_24,cumret_3,cumret_6,cumret_24,logret_12,logret_36,logret_60,cumret_12,cumret_36,cumret_60,termlogret,termcumret,vt+3,me+3,vt+6,me+6,vt+12,me+12,vt+24,me+24,vt+36,me+36,vt+60,me+60,r_t+3,r*_t+3,r_unchg+3,r_chg+3,r_t+6,r*_t+6,r_unchg+6,r_chg+6,r_t+12,r*_t+12,r_unchg+12,r_chg+12,r_t+24,r*_t+24,r_unchg+24,r_chg+24,r_t+36,r*_t+36,r_unchg+36,r_chg+36,r_t+60,r*_t+60,r_unchg+60,r_chg+60,misp,bm
bm,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"(-0.000923, 0.223]",4.968597e+06,25925.023406,2.198999,2.356331,10.439050,0.201639,0.281915,0.533979,0.222120,0.329779,0.813050,0.395669,0.617147,0.727833,0.536531,1.137281,1.959455,2.596468,1735.816439,2.209038,2.361867,2.217406,2.366825,2.230418,2.376182,2.253851,2.392109,2.280184,2.406417,2.326764,2.415391,0.201639,0.249238,0.023834,0.327762,0.281915,0.322774,0.023834,0.436849,0.395669,0.429771,0.023834,0.591629,0.533979,0.581371,0.023834,0.761261,0.617147,0.669982,0.023834,0.838630,0.727833,0.811505,0.023834,0.941195,0.953354,0.055992
"(0.223, 0.335]",7.178936e+06,24213.309829,2.018604,2.109062,10.885133,0.186810,0.258919,0.485218,0.203109,0.299547,0.819824,0.356777,0.560791,0.659948,0.479704,1.101842,1.885860,2.406442,1984.766428,2.023779,2.113374,2.028813,2.115246,2.039342,2.121242,2.059652,2.138052,2.079472,2.149532,2.112227,2.154948,0.186810,0.217752,0.019922,0.281883,0.258919,0.298914,0.019922,0.382870,0.356777,0.407360,0.019922,0.515565,0.485218,0.523805,0.019922,0.637236,0.560791,0.597820,0.019922,0.697390,0.659948,0.716959,0.019922,0.778691,0.796867,0.032259
"(0.335, 0.44]",7.928448e+06,24021.169314,1.858932,1.986131,10.856385,0.179538,0.249458,0.471410,0.198021,0.309672,0.779733,0.341016,0.547521,0.640773,0.486809,1.076758,1.844686,2.330039,1635.480240,1.866325,1.991391,1.873424,1.994890,1.886769,2.004373,1.911257,2.022647,1.935252,2.039879,1.984960,2.056435,0.179538,0.191824,0.019064,0.259380,0.249458,0.274555,0.019064,0.360588,0.341016,0.376559,0.019064,0.490171,0.471410,0.485688,0.019064,0.610116,0.547521,0.560857,0.019064,0.664666,0.640773,0.693951,0.019064,0.751096,0.762558,0.030459
"(0.44, 0.554]",5.422088e+06,24131.326561,1.704679,1.892883,11.101830,0.181751,0.251764,0.476318,0.200384,0.296111,0.787419,0.346204,0.556475,0.650078,0.472226,1.170952,1.965053,2.286911,1456.726040,1.714411,1.898786,1.722970,1.902361,1.738238,1.908162,1.768556,1.933652,1.793835,1.954205,1.845940,1.978703,0.181751,0.178643,0.019552,0.250421,0.251764,0.252344,0.019552,0.345516,0.346204,0.353704,0.019552,0.473315,0.476318,0.461892,0.019552,0.597202,0.556475,0.549516,0.019552,0.663463,0.650078,0.672014,0.019552,0.750095,0.782077,0.032768
"(0.554, 0.677]",6.261803e+06,23930.462710,1.676587,1.901925,11.175262,0.176611,0.245284,0.454363,0.190338,0.292494,0.749661,0.337000,0.534797,0.635951,0.467949,1.086414,2.095829,2.217522,1140.242741,1.684306,1.906670,1.692008,1.911441,1.708728,1.918684,1.736536,1.942039,1.764501,1.964680,1.814034,1.988001,0.176611,0.172553,0.019866,0.242117,0.245284,0.240843,0.019866,0.336445,0.337000,0.342856,0.019866,0.463548,0.454363,0.451320,0.019866,0.582408,0.534797,0.520018,0.019866,0.642951,0.635951,0.649227,0.019866,0.742208,0.794659,0.035724
"(0.677, 0.819]",6.268153e+06,23458.726222,1.602342,1.893563,11.287102,0.172787,0.240538,0.448824,0.189191,0.296332,0.756367,0.331496,0.527510,0.623452,0.472578,1.147473,2.167419,2.212257,801.083451,1.610414,1.900201,1.616706,1.902417,1.630180,1.905600,1.655129,1.920799,1.679368,1.943542,1.739560,1.959711,0.172787,0.179897,0.020510,0.243750,0.240538,0.254217,0.020510,0.337311,0.331496,0.356992,0.020510,0.462472,0.448824,0.448620,0.020510,0.571239,0.527510,0.516987,0.020510,0.634469,0.623452,0.629977,0.020510,0.723288,0.820405,0.040999
"(0.819, 0.994]",5.410697e+06,22474.304930,1.559756,1.897180,11.149166,0.171847,0.237012,0.438531,0.187885,0.288365,0.911413,0.327460,0.503817,0.596697,0.477525,1.217234,2.211312,2.187352,933.391476,1.566624,1.900707,1.573910,1.902472,1.588198,1.903

In [15]:
def ols_coef(x,formula):
    return smf.ols(formula,data=x).fit().params

def fama_macbeth_summary(p):
    s = p.describe().T
    s['std_error'] = s['std']/np.sqrt(s['count'])
    s['tstat'] = s['mean']/s['std_error']
    return s[['mean','std_error','tstat', 'count', 'std']]

def fama_macbeth_reg_panel(regdf, xname='misp', yname='ret', 
                     csname='permno', tsname='jdate_crsp'): 
    # Panel factor setting FM reg
    gamma_cs = (regdf.groupby(tsname).apply(ols_coef,f'{yname} ~ {xname}'))
    gamma_cs = gamma_cs.rename(columns={"Intercept": "alpha_i_t", f"beta_{xname}": "lamba_t"})
    return fama_macbeth_summary(gamma_cs)

In [16]:
%%time
regdf = ret_decomp_dcf5[['jdate_crsp', 'permno', 'ticker', 'me', 'misp']].replace([np.inf, -np.inf], np.nan).dropna()
print(fama_macbeth_reg_panel(regdf, xname='me', yname='misp'))
print('')

               mean  std_error      tstat  count       std
alpha_i_t -2.743510   0.072386 -37.901229  504.0  1.625057
me         0.307259   0.005770  53.249466  504.0  0.129540



In [17]:
%%time
regdf = ret_decomp_dcf5[['jdate_crsp', 'permno', 'ticker', 'bm', 'misp']].replace([np.inf, -np.inf], np.nan).dropna()
print(fama_macbeth_reg_panel(regdf, xname='bm', yname='misp'))
print('')

               mean  std_error      tstat  count       std
alpha_i_t  1.069209   0.013922  76.801577  504.0  0.312542
bm        -0.038724   0.002499 -15.498338  504.0  0.056094

CPU times: user 3.13 s, sys: 51.4 ms, total: 3.18 s
Wall time: 3.18 s
